In [0]:


df1 = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-usage/unified/city-level-usage.v2/fact/").where("granularity='w' and date='2020-02-29' and device_code='ap' and country_code='ae'")

df2 = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-usage/unified/city-level-usage.v2/fact/").where("granularity='w' and date='2020-03-14' and device_code='ap' and country_code='ae'")

df1.createOrReplaceTempView("new_table")
df2.createOrReplaceTempView("old_table")

spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-usage/unified/city-level-usage.v2/fact/").where("granularity='w' and date='2020-02-29' and device_code='ap' and country_code='ae'").show()


In [0]:
%%sh
PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-2-31838298.us-east-1.elb.amazonaws.com -U app_bdp_usage_qa -d cohort -p 7432 << EOF 
select * from plproxy.execute_select(\$proxy\$select date, app_id, city_id, est_average_active_users ,est_average_bytes_per_user
              from mu.city_app_w where city_id='31248510' and app_id='20600003074048' and country_code='ae' and date='2020-03-14'
               \$proxy\$) t (date date, 
              app_id bigint, city_id bigint, est_average_active_users real , est_average_bytes_per_user real) limit 8;
select sum(cnt) from plproxy.execute_select(\$proxy\$select count(1) as cnt from mu.city_app_w where date='2020-03-14' and device_code='ap'  \$proxy\$) t (cnt bigint);

EOF

In [0]:

print spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-usage/unified/city-level-usage.v2/fact/").where("granularity='w' and date='2020-03-14' and device_code='ap'").count()
# 

In [0]:



spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-usage/unified/city-level-usage.v2/fact/").where("granularity='w' and date='2020-03-14' and device_code='ap' and city_id='31248510'").show()

spark.read.parquet("s3://aardvark-prod-pdx-mdm-to-int/city_level/report/routine/version=1.0.0/granularity=weekly/date=2020-03-14/country_code=AE/device_code=android-phone/metric_name=est_average_active_users/part-00000-097c28f4-3b7b-4b9a-8e38-ad16a5b7da53.c000.snappy.parquet").filter("city_id='31248510'").show()


In [0]:
%%sh
aws s3 ls s3://b2c-prod-data-pipeline-unified-usage/unified/city-level-usage.v2/fact/granularity=m/
# aws s3 ls  s3://b2c-prod-data-pipeline-unified-usage/unified/city-level-usage.v1/fact/granularity=w/date=2020-03-07/device_code=ap/country_code=ae/
# aws s3 ls  s3://aardvark-prod-pdx-mdm-to-int/city_level/report/routine/version=1.0.0/granularity=weekly/date=2020-03-14/country_code=AE/device_code=android-phone/metric_name=est_average_active_users/part-00000-097c28f4-3b7b-4b9a-8e38-ad16a5b7da53.c000.snappy.parquet











In [0]:

def get_compared_data(date):
    df_new = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-usage/unified/city-level-usage.v2/fact/").where("granularity='w' and date='{}'".format(date)).filter("device_code='ap'")
    df_old = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-usage/unified/city-level-usage.v1/fact/granularity=w/date={}/device_code=ap/".format(date))

    list1= df_new.sample(False, 0.001, seed=0).limit(20).take(20)
    return list1, df_old

test_list, df_old = get_compared_data('2020-02-29')



def compare(test_list, df_old):
    columns_list = df_old.columns

    for x in range(20):
        list2 = df_old.filter("app_id='{}' and city_id='{}'".format(test_list[x]["app_id"],test_list[x]["city_id"])).collect()
        # print list2
        for column in columns_list:
            if test_list[x][column] != list2[0][column]:
                print 'not equal',column, test_list[x][column], list2[0][column]
    print 'Finished'           

compare(test_list, df_old)

# print spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-usage/unified/city-level-usage.v2/fact/").where("granularity='m' and date='2020-02-29'").filter("device_code='ap'").count()

In [0]:


df1 = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-usage/unified/city-level-usage.v2/fact/").where("granularity='w' and date='2020-02-29' and device_code='ap' and country_code='ae'")

df2 = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-usage/unified/city-level-usage.v2/fact/").where("granularity='w' and date='2020-02-22' and device_code='ap' and country_code='ae'")

df1.createOrReplaceTempView("new_table")
df2.createOrReplaceTempView("old_table")

spark.sql("select city_id, app_id from (select city_id, app_id from old_table intersect select city_id, app_id from new_table) as prod").show()

In [0]:


from aadatapipelinecore.core.urn import Urn
from aadatapipelinecore.core.pipeline.type_ import DataType, EventType, ManipulationType
from aadatapipelinecore.core.fs.driver import unified

# namespace='aa.city-level-usage.v1'
urn = Urn(
        namespace="aa.city-level-usage.v2",
        event="transform",
        data_type=DataType.FACT
    )

df = unified.read(spark, urn,  sql_where="granularity='m' and date='2018-09-30'").data_dto_list[0].dataframe
v2_data_collect = df.filter("app_id='20600007997953'").collect()
print v2_data_collect
# print data.take(1)
# t =data.take(2)
# print [x for x in schema]
# keys = {"city_id", "app_id","est_average_active_users","est_usage_penetration", "est_average_session_per_user","est_average_time_per_user","est_average_bytes_per_user","est_total_time","est_average_bytes_per_session","est_average_session_duration","est_affinity","device_code","country_code"}
# print t[0]

# for x in keys:
#     print t[0][x]


In [0]:


df2 = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-usage/unified/city-level-usage.v1/fact/granularity=m/date=2018-09-30/").cache()
v1_data_collect= df2.filter("app_id='20600007997953'").collect()




In [0]:

# data_v2=changed_data.take(1)

for x in changed_schema:
    if x not in schema:
        print x

for x in range(0,len(v1_data_collect)):
    for k in keys:
        if v2_data_collect[x][k] != v1_data_collect[x][k]:
            print  v2_data_collect[x][k] , v1_data_collect[x][k]


In [0]:

from aadatapipelinecore.core.log import logger

import unittest
from pyspark.sql.utils import AnalysisException
from aadatapipelinecore.core.fs.device import s3
from aadatapipelinecore.core.fs import Conf

# from aadatapipelinecore.core.fs.device import meta_bucket
# df2 = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-usage/unified/city-level-usage.v1/fact/granularity=m/date=2018-09-30/").cache()


# s3_bucket_list = s3.S3Bucket(Conf(bucket_name='b2c-prod-data-pipeline-unified-usage'))
# path = s3_bucket_list.all(prefix="unified/city-level-usage.v1/fact/granularity=m", depth_is_1=True)
print path[0].split("=")[-1].strip("/")
test = [(x.split("=")[-1].strip("/"),'a') for x in path ]
test.extend([(u'2018-01-31', 'c'), (u'2018-02-28', 'ca'), (u'2018-03-31', 'a'), (u'2018-04-30', 'a'), (u'2018-05-31', 'a'), (u'2018-06-30', 'a'), (u'2018-07-31', 'a'), (u'2018-08-31', 'a'), (u'2018-09-30', 'a'), (u'2018-10-31', 'a'), (u'2018-11-30', 'a'), (u'2018-12-31', 'a'), (u'2019-01-31', 'a'), (u'2019-02-28', 'a'), (u'2019-03-31', 'a'), (u'2019-04-30', 'a'), (u'2019-05-31', 'a'), (u'2019-06-30', 'a'), (u'2019-07-31', 'a'), (u'2019-08-31', 'ac'), (u'2019-09-30', 'ac'), (u'2019-10-31', 'ac'), (u'2019-11-30', 'a'), (u'2019-12-31', 'a'), (u'2020-01-31', 'a')])
print len(test)

In [0]:

from aadatapipelinecore.core.fs.device import s3
from aadatapipelinecore.core.urn import Urn
from aadatapipelinecore.core.pipeline.type_ import DataType, EventType, ManipulationType
from aadatapipelinecore.core.fs.driver import unified

granularity_list = ['m','w']

def get_path_list(granularity):
    s3_bucket_list = s3.S3Bucket(Conf(bucket_name='b2c-prod-data-pipeline-unified-usage'))
    path = s3_bucket_list.all(prefix="unified/city-level-usage.v1/fact/granularity={}".format(granularity), depth_is_1=True)
    # print path[0].split("=")[-1].strip("/")
    # print [x for x in path[x].split("=")[-1].strip("/") ]
    date_list = [(granularity, x.split("=")[-1].strip("/")) for x in path ]
    return date_list
    


def get_delta_lake_version(granularity, date):
    urn = Urn(
            namespace="aa.city-level-usage.v2",
            event="transform",
            data_type=DataType.FACT
        )

    df_v2 = unified.read(spark, urn,  sql_where="granularity='{}' and date='{}'".format(granularity, date)).data_dto_list[0].dataframe
    v2_data_count = df_v2.count()
    return v2_data_count
    
def get_old_version(granularity, date):
    df_v1 = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-usage/unified/city-level-usage.v1/fact/granularity={}/date={}/".format(granularity, date))
    return df_v1.count()
    
def compare(granularity, date):
    count1 = get_delta_lake_version(granularity, date)
    count2 = get_old_version(granularity, date)
    if count1 != count2:
        print 'Not equal!!!!! v1 : {}, v2 : {}, granularity: {}, date: {}'.format(count1, count2, granularity, date)
    else:
        print 'v1 : {}, v2 : {}, granularity: {}, date: {}'.format(count1, count2, granularity, date)
 
test_path = []
for g in granularity_list:
    test_path.extend(get_path_list(g))

for case in test_path:
    print case
    compare(case[0],case[1])


In [0]:

from aadatapipelinecore.core.urn import Urn
from aadatapipelinecore.core.utils.runtime import is_prod
from aadatapipelinecore.core.pipeline.type_ import DataType, EventType, ManipulationType

from aadatapipelinecore.core.fs.driver import unified
urn = Urn(
        namespace="city-level-usage-v1.usage",
        event="transform",
        data_type=DataType.FACT
    )
result =  unified.read(spark, urn, "")


print result.urn